# Laboratorio 1: Aprendizaje No Supervisado
Reglas de asociación entre películas

*Integrantes:

- Alain, Kraupl
- Carlos Piaggio
- Juan Pablo Analian

Obtener reglas de asociación entre películas en el dataset movielens como si fuera recomendación!
Aplicar diferentes métricas de ordenamiento. Hacer un pequeño informe

In [1]:
import numpy as np
import pandas as pd
from itertools import combinations, groupby
from collections import Counter
from efficient_apriori import apriori

In [2]:
df_movies = pd.read_csv('./movies.csv')
df_rat = pd.read_csv('./ratings.csv' ,nrows=1000000) #menos de 10-seg en levantar las 20 millones

In [3]:
print('movies', len(df_movies))
display(df_movies.head(10))
print('ratings', len(df_rat))
display(df_rat.head(10))

movies 27278


,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy
5,6,Heat (1995),Action|Crime|Thriller
6,7,Sabrina (1995),Comedy|Romance
7,8,Tom and Huck (1995),Adventure|Children
8,9,Sudden Death (1995),Action
9,10,GoldenEye (1995),Action|Adventure|Thriller


ratings 1000000


,userId,movieId,rating,timestamp
0,1,2,3.5,1112486027
1,1,29,3.5,1112484676
2,1,32,3.5,1112484819
3,1,47,3.5,1112484727
4,1,50,3.5,1112484580
5,1,112,3.5,1094785740
6,1,151,4.0,1094785734
7,1,223,4.0,1112485573
8,1,253,4.0,1112484940
9,1,260,4.0,1112484826


In [4]:
df_rat.rating.value_counts().sort_index(ascending=False)

5.0    144891
4.5     75766
4.0    281246
3.5    108047
3.0    216118
2.5     42948
2.0     71249
1.5     13369
1.0     33588
0.5     12778
Name: rating, dtype: int64

In [28]:
print("Cantidad de registros con rating >= a 4:",len(df_rat[df_rat.rating >= 4.0]))
df_rat[df_rat.rating >= 4.0].rating.value_counts()

Cantidad de registros con rating >= a 4: 501903


4.0    281246
5.0    144891
4.5     75766
Name: rating, dtype: int64

In [29]:
df_rat.loc[df_rat.rating >= 4.0, ['userId','movieId']].head(5)

,userId,movieId
6,1,151
7,1,223
8,1,253
9,1,260
10,1,293


In [30]:
df_tran = pd.merge(df_rat.loc[df_rat.rating >= 4.0, ['userId','movieId']], df_movies[['movieId','title']] ,on='movieId', how= "inner") 
df_tran = df_tran.sort_values(by='userId', axis=0, ascending=True, inplace=False, kind='quicksort', na_position='last')
display(df_tran.head(20))
#df_tran.groupby(['title']).agg({'userId':'count'}).sort_values('userId', axis=0, ascending=False).head(10)

,userId,movieId,title
0,1,151,Rob Roy (1995)
41255,1,5026,"Brotherhood of the Wolf (Pacte des loups, Le) ..."
41324,1,5039,Dragonslayer (1981)
37907,1,4128,"Lost Boys, The (1987)"
41340,1,5171,"Time Machine, The (2002)"
41377,1,5540,Clash of the Titans (1981)
41407,1,5797,"Company of Wolves, The (1984)"
41414,1,5816,Harry Potter and the Chamber of Secrets (2002)
30259,1,2542,"Lock, Stock & Two Smoking Barrels (1998)"
28129,1,2118,"Dead Zone, The (1983)"


In [31]:
df_tran2 = df_tran.values[:,[0,2]]
print(df_tran2[0:10])

[[1 'Rob Roy (1995)']
 [1 'Brotherhood of the Wolf (Pacte des loups, Le) (2001)']
 [1 'Dragonslayer (1981)']
 [1 'Lost Boys, The (1987)']
 [1 'Time Machine, The (2002)']
 [1 'Clash of the Titans (1981)']
 [1 'Company of Wolves, The (1984)']
 [1 'Harry Potter and the Chamber of Secrets (2002)']
 [1 'Lock, Stock & Two Smoking Barrels (1998)']
 [1 'Dead Zone, The (1983)']]


In [91]:
transactions=[]
for user_id, movie in groupby(df_tran2, lambda x: x[0]):
    transactions.append([item[1] for item in movie])

print("Las películas vistas por el userID = 1 >", len(transactions[0]))

Las películas vistas por el userID = 1 > 88


In [92]:
itemsets, rules = apriori(transactions, min_support=0.20,  min_confidence=0.15)

In [93]:
print("itemset len:",len(itemsets))
print("rules len:",len(rules))

itemset len: 2
rules len: 16


In [94]:
print(rules[0])

{Shawshank Redemption, The (1994)} -> {Forrest Gump (1994)} (conf: 0.525, supp: 0.213, lift: 1.482, conv: 1.360)


In [95]:
print(rules[0].lhs, type(rules[0].lhs))
print(rules[0].rhs)
print(rules[0].lift)
print(rules[0].support) 
print(rules[0].confidence) 
print(rules[0].num_transactions)

('Shawshank Redemption, The (1994)',) <class 'tuple'>
('Forrest Gump (1994)',)
1.4816922860509087
0.21283231843160552
0.5252932551319648
6733


In [81]:
#ordenados por el de mayor LIFT 
for rule in sorted(rules, key=lambda rule: rule.lift,reverse=True):
    print(rule)

{Star Wars: Episode V - The Empire Strikes Back (1980)} -> {Star Wars: Episode IV - A New Hope (1977)} (conf: 0.821, supp: 0.217, lift: 2.568, conv: 3.809)
{Star Wars: Episode IV - A New Hope (1977)} -> {Star Wars: Episode V - The Empire Strikes Back (1980)} (conf: 0.679, supp: 0.217, lift: 2.568, conv: 2.293)
{Star Wars: Episode VI - Return of the Jedi (1983)} -> {Star Wars: Episode IV - A New Hope (1977)} (conf: 0.815, supp: 0.202, lift: 2.548, conv: 3.681)
{Star Wars: Episode IV - A New Hope (1977)} -> {Star Wars: Episode VI - Return of the Jedi (1983)} (conf: 0.631, supp: 0.202, lift: 2.548, conv: 2.039)
{Usual Suspects, The (1995)} -> {Pulp Fiction (1994)} (conf: 0.690, supp: 0.207, lift: 1.831, conv: 2.012)
{Pulp Fiction (1994)} -> {Usual Suspects, The (1995)} (conf: 0.549, supp: 0.207, lift: 1.831, conv: 1.553)
{Silence of the Lambs, The (1991)} -> {Pulp Fiction (1994)} (conf: 0.641, supp: 0.230, lift: 1.699, conv: 1.734)
{Pulp Fiction (1994)} -> {Silence of the Lambs, The (1991

In [83]:
columns = ['lhs', 'lhs_count', 'rhs','rhs_count', 'support', 'lift', 'confidence']
list_of_rules=[]
for r in rules:
    list_of_rules.append([r.lhs, len(r.lhs), r.rhs, len(r.rhs), r.support,r.lift, r.confidence])

df_res = pd.DataFrame(list_of_rules, columns=columns)


In [86]:
df_res.sort_values(by='support', ascending=False)

,lhs,lhs_count,rhs,rhs_count,support,lift,confidence
2,"(Shawshank Redemption, The (1994),)",1,"(Pulp Fiction (1994),)",1,0.233328,1.527136,0.575880
3,"(Pulp Fiction (1994),)",1,"(Shawshank Redemption, The (1994),)",1,0.233328,1.527136,0.618748
4,"(Silence of the Lambs, The (1991),)",1,"(Pulp Fiction (1994),)",1,0.229912,1.699104,0.640728
5,"(Pulp Fiction (1994),)",1,"(Silence of the Lambs, The (1991),)",1,0.229912,1.699104,0.609689
8,"(Silence of the Lambs, The (1991),)",1,"(Shawshank Redemption, The (1994),)",1,0.226942,1.560956,0.632450
9,"(Shawshank Redemption, The (1994),)",1,"(Silence of the Lambs, The (1991),)",1,0.226942,1.560956,0.560117
12,(Star Wars: Episode V - The Empire Strikes Bac...,1,"(Star Wars: Episode IV - A New Hope (1977),)",1,0.217288,2.567694,0.821449
13,"(Star Wars: Episode IV - A New Hope (1977),)",1,(Star Wars: Episode V - The Empire Strikes Bac...,1,0.217288,2.567694,0.679201
0,"(Shawshank Redemption, The (1994),)",1,"(Forrest Gump (1994),)",1,0.212832,1.481692,0.525293
1,"(Forrest Gump (1994),)",1,"(Shawshank Redemption, The (1994),)",1,0.212832,1.481692,0.600335


In [88]:
df_res.sort_values(by='confidence', ascending=False)

,lhs,lhs_count,rhs,rhs_count,support,lift,confidence
12,(Star Wars: Episode V - The Empire Strikes Bac...,1,"(Star Wars: Episode IV - A New Hope (1977),)",1,0.217288,2.567694,0.821449
14,(Star Wars: Episode VI - Return of the Jedi (1...,1,"(Star Wars: Episode IV - A New Hope (1977),)",1,0.201842,2.548278,0.815237
6,"(Usual Suspects, The (1995),)",1,"(Pulp Fiction (1994),)",1,0.207040,1.830933,0.690441
13,"(Star Wars: Episode IV - A New Hope (1977),)",1,(Star Wars: Episode V - The Empire Strikes Bac...,1,0.217288,2.567694,0.679201
10,"(Usual Suspects, The (1995),)",1,"(Shawshank Redemption, The (1994),)",1,0.201990,1.662520,0.673601
4,"(Silence of the Lambs, The (1991),)",1,"(Pulp Fiction (1994),)",1,0.229912,1.699104,0.640728
8,"(Silence of the Lambs, The (1991),)",1,"(Shawshank Redemption, The (1994),)",1,0.226942,1.560956,0.632450
15,"(Star Wars: Episode IV - A New Hope (1977),)",1,(Star Wars: Episode VI - Return of the Jedi (1...,1,0.201842,2.548278,0.630919
3,"(Pulp Fiction (1994),)",1,"(Shawshank Redemption, The (1994),)",1,0.233328,1.527136,0.618748
5,"(Pulp Fiction (1994),)",1,"(Silence of the Lambs, The (1991),)",1,0.229912,1.699104,0.609689


In [87]:
df_res.sort_values(by='lift', ascending=False)

,lhs,lhs_count,rhs,rhs_count,support,lift,confidence
12,(Star Wars: Episode V - The Empire Strikes Bac...,1,"(Star Wars: Episode IV - A New Hope (1977),)",1,0.217288,2.567694,0.821449
13,"(Star Wars: Episode IV - A New Hope (1977),)",1,(Star Wars: Episode V - The Empire Strikes Bac...,1,0.217288,2.567694,0.679201
14,(Star Wars: Episode VI - Return of the Jedi (1...,1,"(Star Wars: Episode IV - A New Hope (1977),)",1,0.201842,2.548278,0.815237
15,"(Star Wars: Episode IV - A New Hope (1977),)",1,(Star Wars: Episode VI - Return of the Jedi (1...,1,0.201842,2.548278,0.630919
6,"(Usual Suspects, The (1995),)",1,"(Pulp Fiction (1994),)",1,0.207040,1.830933,0.690441
7,"(Pulp Fiction (1994),)",1,"(Usual Suspects, The (1995),)",1,0.207040,1.830933,0.549035
4,"(Silence of the Lambs, The (1991),)",1,"(Pulp Fiction (1994),)",1,0.229912,1.699104,0.640728
5,"(Pulp Fiction (1994),)",1,"(Silence of the Lambs, The (1991),)",1,0.229912,1.699104,0.609689
10,"(Usual Suspects, The (1995),)",1,"(Shawshank Redemption, The (1994),)",1,0.201990,1.662520,0.673601
11,"(Shawshank Redemption, The (1994),)",1,"(Usual Suspects, The (1995),)",1,0.201990,1.662520,0.498534
